In [2]:
from senta import Senta
from mytool import progress_bar
senta = Senta()

from pyecharts.charts import Pie, Timeline
from pyecharts.globals import ThemeType
import pyecharts.options as opts

tl = Timeline(
    init_opts=opts.InitOpts(width='1400px',
                            height='1000px',
                            page_title="微博热榜",
                            theme=ThemeType.INFOGRAPHIC)
)
data = []
import os
for root, dirs, files in os.walk('./out/hotrank/'):
    pb = progress_bar(len(files))
    cnt = 0
    for file in files:
        with open(root+file, 'r', encoding='utf-8') as f:
            for line in f.readlines():
                senta.add(line.split(',')[0])
        senta.predict()
        num_pos = 0
        num_neg = 0
        num_neu = 0
        for item in senta.results:
            positive_prob = round(item['positive_probs'], 2)
            negative_prob = round(item['negative_probs'], 2)
            if positive_prob >= 0.7:
                num_pos += 1
            elif negative_prob >= 0.6:
                num_neg += 1
            else:
                num_neu += 1
        data.append({f"{file[:5]} {file[6:8]}:{file[9:11]}":[num_pos,num_neu,num_neg]})
        pie = (
            Pie()
            .add(
                series_name="舆情分布",
                data_pair=[["积极",num_pos],["中性",num_neu],["消极",num_neg]],
                rosetype="radius",
                radius="55%",
                center=["50%", "50%"],
                label_opts=opts.LabelOpts(
                    position="outside",
                    formatter="{a|{a}}{abg|}\n{hr|}\n {b|{b}: }{c}  {per|{d}%}  ",
                    background_color="#eee",
                    border_color="#aaa",
                    border_width=1,
                    border_radius=4,
                    rich={
                        "a": {"color": "#999", "lineHeight": 22, "align": "center"},
                        "abg": {
                            "backgroundColor": "#e3e3e3",
                            "width": "100%",
                            "align": "right",
                            "height": 22,
                            "borderRadius": [4, 4, 0, 0],
                        },
                        "hr": {
                            "borderColor": "#aaa",
                            "width": "100%",
                            "borderWidth": 0.5,
                            "height": 0,
                        },
                        "b": {"fontSize": 16, "lineHeight": 33},
                        "per": {
                            "color": "#eee",
                            "backgroundColor": "#334455",
                            "padding": [2, 4],
                            "borderRadius": 2,
                        },
                    },
                )
            )
            .set_global_opts(
                title_opts=opts.TitleOpts(
                    title="Customized Pie",
                    pos_left="center",
                    pos_top="20",
                    title_textstyle_opts=opts.TextStyleOpts(color="#fff"),
                ),
                legend_opts=opts.LegendOpts(is_show=False),
            )
            .set_series_opts(
                tooltip_opts=opts.TooltipOpts(
                    trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"
                ),
            )
        )
        tl.add(pie,f"{file[:5]} {file[6:8]}:{file[9:11]}")
        pb.progress(cnt)
        cnt += 1
tl.render('./out/tmp/hotrank_senti.html')

D:\anaconda3\envs\torch_ernie\lib\site-packages\pip\_vendor\packaging\version.py:130: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  DeprecationWarning,
D:\anaconda3\envs\torch_ernie\lib\site-packages\pip\_vendor\packaging\version.py:130: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  DeprecationWarning,
[2021-06-13 14:23:50,991] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object
[2021-06-13 14:23:53,818] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object
[ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo](23608/23609)loaded...

'd:\\CS50\\Final\\out\\tmp\\hotrank_senti.html'

In [3]:
import json
with open('./out/hotrank_stat.json','w',encoding='utf-8') as f:
    json.dump(data,f)

In [4]:
from pyecharts.charts import Bar, Pie, Timeline, Grid
from pyecharts.globals import ThemeType
import pyecharts.options as opts
tl = Timeline(
    init_opts=opts.InitOpts(width='1400px',
                            height='1000px',
                            page_title="微博热榜",
                            theme=ThemeType.INFOGRAPHIC)
)

import os
for root, dirs, files in os.walk('./out/hotrank/'):
    for file in files:
        xdata = []
        ydata = []
        with open(root+file,'r', encoding='utf-8') as f:
            for line in f.readlines():
                xdata.append(line.split(',')[0])
                ydata.append(line.split(',')[1])
        rank = (
            Bar()
            .add_xaxis(xdata[::-1])
            .add_yaxis('热度',ydata[::-1])
            .reversal_axis()
            .set_global_opts(
                title_opts=opts.TitleOpts(title="热榜",subtitle=f"{file[:5]} {file[6:8]}:{file[9:11]}"),
                yaxis_opts=opts.AxisOpts(name="热榜条目"),
                xaxis_opts=opts.AxisOpts(name="热度"),
            )
            .set_series_opts(label_opts=opts.LabelOpts(position="right"))
        )
        tl.add(rank,f"{file[:5]} {file[6:8]}:{file[9:11]}")
tl.render('./out/tmp/hotrank.html')

'd:\\CS50\\Final\\out\\tmp\\hotrank.html'